In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practise6').getOrCreate()

23/02/15 02:29:30 WARN Utils: Your hostname, Sujeets-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.122 instead (on interface en0)
23/02/15 02:29:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/15 02:29:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/15 02:29:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/15 02:29:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/15 02:29:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/15 02:29:32 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [8]:
training = spark.read.csv('test4.csv', header=True, inferSchema=True)

In [9]:
training.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
| Sujeet| 22|         3| 35000|
|  Ajeet| 18|         2| 26000|
|  Akash| 14|         4| 20000|
|  Manoj| 33|         1| 20000|
|Manisha| 37|         8| 15000|
|  Chunu|  5|         7| 18000|
+-------+---+----------+------+



In [10]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

### [Age, Experience] -----> New Feature -----> Independent Feature

In [14]:
from pyspark.ml.feature import VectorAssembler

In [21]:
featureassembler = VectorAssembler(inputCols=['Age', 'Experience'], outputCol='Independent Features')

In [23]:
output = featureassembler.transform(training)

In [24]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|Age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
| Sujeet| 22|         3| 35000|          [22.0,3.0]|
|  Ajeet| 18|         2| 26000|          [18.0,2.0]|
|  Akash| 14|         4| 20000|          [14.0,4.0]|
|  Manoj| 33|         1| 20000|          [33.0,1.0]|
|Manisha| 37|         8| 15000|          [37.0,8.0]|
|  Chunu|  5|         7| 18000|           [5.0,7.0]|
+-------+---+----------+------+--------------------+



In [25]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [26]:
finalized_data = output.select("Independent Features", "Salary")

In [27]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [22.0,3.0]| 35000|
|          [18.0,2.0]| 26000|
|          [14.0,4.0]| 20000|
|          [33.0,1.0]| 20000|
|          [37.0,8.0]| 15000|
|           [5.0,7.0]| 18000|
+--------------------+------+



In [28]:
from pyspark.ml.regression import LinearRegression

#train_test_split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])

regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

23/02/15 02:48:41 WARN Instrumentation: [80156672] regParam is zero, which might cause numerical instability and overfitting.
23/02/15 02:48:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/15 02:48:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/02/15 02:48:41 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [29]:
regressor.coefficients

DenseVector([-184.3104, -857.5322])

In [30]:
regressor.intercept

28165.660236199456

In [31]:
pred_results = regressor.evaluate(test_data)

In [32]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|           [5.0,7.0]| 18000|21241.382995778353|
|          [22.0,3.0]| 35000| 21538.23545129055|
+--------------------+------+------------------+



In [35]:
pred_results.meanSquaredError, pred_results.meanAbsoluteError

(95862834.2451058, 8351.5737722439)